In [1]:
import pandas as pd
import numpy as np
#import eals
#import scipy.sparse import csr_matrix

In [2]:
from eals import ElementwiseAlternatingLeastSquares, load_model

In [3]:
from cv_py import CrossValidation

In [4]:
import implicit
from implicit.datasets.movielens import get_movielens
movies1, ratings1 = get_movielens("1M")
ratings_t = ratings1.T.tocsr()

In [5]:
from implicit.evaluation import train_test_split
from implicit.evaluation import ranking_metrics_at_k

In [6]:
ratings_t_train, ratings_t_test = train_test_split(ratings_t, train_percentage = 0.2)

In [7]:
cv = CrossValidation(ratings_t_train, 5)

In [8]:
test, train = cv.split_k_fold()

In [9]:
space = {'factors' : [64], 'alpha': [0.5], 'regularization':[0.02], 'w0':[10]}
test_cv_eals = cv.hyperp_tuning(test, train, space, 'eALS', 'full')
test_cv_eals

,factors,alpha,regularization,w0,precision,map,ndcg,auc,mpr
0,64.0,0.5,0.02,10.0,0.034422,0.013265,0.029070,0.516533,0.303518
1,NaN,NaN,NaN,NaN,0.032903,0.012202,0.027433,0.515599,0.301822
2,NaN,NaN,NaN,NaN,0.033910,0.012106,0.027506,0.515742,0.302853
3,NaN,NaN,NaN,NaN,0.032681,0.011526,0.026419,0.514858,0.298932
4,NaN,NaN,NaN,NaN,0.033331,0.012562,0.027881,0.515154,0.298221


,factors,alpha,regularization,w0,precision,map,ndcg,auc,mpr
0,64,0.5,0.02,10,0.032621,0.012702,0.02778,0.515708,0.302857


In [11]:
space = {'factors' : [64], 'alpha': [10], 'regularization':[0.02], 'iterations':[10]}
test_cv_ials = cv.hyperp_tuning(test, train, space, 'iALS', 'full')
test_cv_ials

,factors,alpha,regularization,iterations,precision,map,ndcg,auc,mpr
0,64.0,10.0,0.02,10.0,0.040677,0.016087,0.035094,0.521014,0.273809
1,NaN,NaN,NaN,NaN,0.040230,0.015158,0.033598,0.519890,0.271321
2,NaN,NaN,NaN,NaN,0.037694,0.014155,0.031512,0.518963,0.274474
3,NaN,NaN,NaN,NaN,0.041219,0.015366,0.034285,0.520397,0.272043
4,NaN,NaN,NaN,NaN,0.041223,0.016261,0.035410,0.520609,0.271625


In [13]:
model_ials = implicit.als.AlternatingLeastSquares(factors=64, alpha=10, regularization=0.02, iterations=10)
model_ials.fit(ratings_t_train)
ranking_metrics_at_k(model_ials, ratings_t_train, ratings_t_test, 5)

100%|██████████| 6040/6040 [00:00<00:00, 10200.44it/s]


{'precision': 0.4492384105960265,
 'map': 0.3525491169977911,
 'ndcg': 0.4610603607518856,
 'auc': 0.5140829636724776}

In [11]:
model = ElementwiseAlternatingLeastSquares(factors=64, alpha=0.5, regularization=0.02, w0=10)
model

In [13]:
model.fit(ratings_t_train)

In [18]:
model_ials = implicit.als.AlternatingLeastSquares()
model_ials.fit(ratings_t_train)

100%|██████████| 15/15 [00:04<00:00,  3.19it/s]


In [34]:
model.user_factors.shape

(6041, 100)

In [15]:
model_ials.user_factors = model.user_factors
model_ials.item_factors = model.item_factors

In [16]:
ranking_metrics_at_k(model_ials, ratings_t_train, ratings_t_test, 5)

100%|██████████| 6040/6040 [00:00<00:00, 9108.16it/s]


{'precision': 0.3729139072847682,
 'map': 0.28180518763796764,
 'ndcg': 0.3845867795300777,
 'auc': 0.5106152646123584}

In [63]:
user_list = ratings_counts_test.userId
test_mpr = get_MPR(user_list)
test_mpr

0.4206324569991457

In [80]:
def get_MPR(users, model) :
    zähler = 0
    nenner = 0
    for u in users :
        predictions = get_rec_list(u, model)
        counts = get_counts(u)
        df = pd.DataFrame({'predictions': predictions, 'counts' : counts})
        df = df.sort_values('predictions', ascending=False)
        df['rank'] = get_rank(u)
        for i in range(len(df)) :
            zähler += df['counts'].iloc[i] * df['rank'].iloc[i]
            nenner += df['counts'].iloc[i] 
    return zähler / nenner

In [81]:
# helperfunc for get_mpr
def get_rec_list(user, model) :
    items = ratings_counts_test[ratings_counts_test.userId == user].genreId.unique()
    ratings = []
    for i in items :
        pred = np.inner(model.user_factors[user], model.item_factors[i])
        ratings.append(pred)
    return ratings

In [29]:
# helperfunc for get_mpr
def get_rank(user) :
    no_of_items = ratings_counts_test[ratings_counts_test.userId == user].genreId.nunique()
    percentlile_rank = [] 
    for i in range(no_of_items) :
        percentlile_rank.append((i+1) / no_of_items)
    return percentlile_rank

In [30]:
# helperfunc for get_mpr
def get_counts(user) :
    no_of_items = ratings_counts_test[ratings_counts_test.userId == user].counts
    return no_of_items

In [35]:
rmse_test = evaluate(ratings_counts_test, 'mae')
rmse_test

In [194]:
def evaluate(test_df, metric) :
    squared_diff = 0
    abs_diff = 0
    num_obs = 0
    for i in test_df.index:
        r = test_df.counts[i]
        user_id = test_df.userId[i]
        item_id = test_df.genreId[i]
        r_hat = np.inner(model.user_factors[user_id], model.item_factors[item_id])
        squared_diff += (r - r_hat) ** 2
        abs_diff += abs(r - r_hat)
        num_obs += 1
    if metric == 'rmse' :
        return np.sqrt(squared_diff / num_obs)
    if metric == 'mse' :
        return squared_diff / num_obs
    if metric == 'mae' :
        return abs_diff / num_obs
    

In [188]:
def evaluate(test_df, rating_col, user_col, item_col, metric) :
    squared_diff = 0
    abs_diff = 0
    num_obs = 0
    for i in test_df.index:
        r = test_df[rating_col].iloc[i]
        user_id = test_df[user_col].iloc[i]
        item_id = test_df[item_col].iloc[i]
        r_hat = np.inner(model.user_factors[user_id], model.item_factors[item_id])
        squared_diff += (r - r_hat) ** 2
        abs_diff += abs(r - r_hat)
        num_obs += 1
    if metric == 'rmse' :
        return np.sqrt(squared_diff / num_obs)
    if metric == 'mse' :
        return squared_diff / num_obs
    if metric == 'mae' :
        return abs_diff / num_obs
    


In [2]:
from implicit.evaluation import AUC_at_k

In [6]:
!pip install --upgrade implicit

Requirement already up-to-date: implicit in c:\users\chris\anaconda3\lib\site-packages (0.5.2)


In [7]:
import implicit

In [9]:
model_ials = implicit.als.AlternatingLeastSquares(factors=100, regularization=10, iterations=30)

In [10]:
ratings_counts_train_coo = ratings_counts_train_sparse.tocoo()

NameError: name 'ratings_counts_train_sparse' is not defined

In [94]:
model_ials.fit(ratings_counts_train_coo)

100%|██████████| 30/30 [00:00<00:00, 46.14it/s]


In [95]:
user_list = ratings_counts_test.userId
test_mpr = get_MPR(user_list, model_ials)
test_mpr

0.4174168437641856